In [7]:
# Imports
import json
import requests
import requests.exceptions
import pandas as pd
import numpy as np
import sqlite3

In [8]:
# Benchmark i1: Fetch/download/retrieve a remote data file by URL (API call)
header_var ={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
## API found at https://mixedanalytics.com/blog/list-actually-free-open-no-auth-needed-apis/
## This specific API allows access to congressmembers' (specifically, Representatives from the House of Representatives) stock purchase data.
## The data include the name of the representative, the district he/she represents, the stock ticker, the transaction date, the disclosure date and year, the amount of money spent, etc. 
url1 = 'https://house-stock-watcher-data.s3-us-west-2.amazonaws.com/data/all_transactions.json'

# Benchmark ii:  produce informative errors should it be unable to complete an operation. (Try/Catch with error messages)
try:
    response1 = requests.request("GET", url1, headers=header_var)
    response1.raise_for_status()
except requests.exceptions.HTTPError as errh:
    print("An Http Error occurred. This stock does not exist: " + repr(errh))
except requests.exceptions.ConnectionError as errc:
    print("Connection error. An Error Connecting to the API occurred:" + repr(errc))
except requests.exceptions.Timeout as errt:
    print( "Timeout error. A Timeout Error occurred: " + repr(errt))
except requests.exceptions.RequestException as err:
    print("An Unknown Error occurred:" + repr(err))
json1 = response1.json()

In [9]:
# Store the results locally in JSON format
with open('project.json', 'w') as outfile:
    json.dump(json1, outfile)
json.dumps(json1, indent=4)
print('File sucessfully downloaded as: project.json')

File sucessfully downloaded as: project.json


In [10]:
print("What would you like to do?")
print("1. View dataset")
print("2. Select a representative")
print("3. Select a stock")
print("4. Select a year")
print("5. Select a district")
print("6. View all representatives")
print("7. View all stocks")
print("8. View all years")
print("9. View all districts")

What would you like to do?
1. View dataset
2. Select a representative
3. Select a stock
4. Select a year
5. Select a district
6. View all representatives
7. View all stocks
8. View all years
9. View all districts


In [27]:
##Ask user to select from the above actions
action = str(input("Please enter a number 1 through 9 to select an action:"))
df = pd.read_json('project.json')
df.pop("ptr_link") #Benchmark i3: Modify the number of columns from the source to the destination (remove the https://disclosures-clerk.house.gov/ links to specific representative donations because the links were expired)
##If user requests to view the full dataset...
if action=='1':
    print(df) ##Output full dataset regardless of whether or not there is an error in outputting the file to csv or sql
    shape=df.shape #Benchmark i5: output brief summary of the data file ingestion after it has processed and include number of records and columns
    print('Number of columns:', shape[1])
    print('Number of records:', shape[0])
    #Benchmark i2: Convert the general format and data structure of the data source to any target (user selects between csv and sql output type)
    output_type = str(input("Output csv or sql?:"))
    if output_type=="csv": ##If the user selects output as csv
        df.to_csv("representative_stock_data.csv") #Benchmark i4: Converted (new) file is written to disk
        print("File sucessfully downloaded as: representative_stock_data.csv") #Confirms output success
    elif output_type=="sql": ##If the user selects output as sql
        conn=sqlite3.connect("representative_stock_data.db")
        cur=conn.cursor()
        cur.execute("CREATE TABLE IF NOT EXISTS representative_stock_data (disclosure_year number, disclosure_date text, transaction_date text, owner text, ticker text, asset_description text, type text, amount text, representative text, district text, ptr_link text, cap_gains_over_200_usd number)")
        conn.commit()
        df.to_sql("shows", conn, if_exists="replace", index=False)
        cur.execute('''
        SELECT * FROM representative_stock_data
        ''') #Benchmark i4: Converted (new) file is written to disk
        print("File sucessfully downloaded as: representative_stock_data.sql") #Confirms output success
    else:
        print("Error outputting. Please enter 'csv' or 'sql'. Check your capitalization and spelling") #Output error notification
##If user requests to view specific representatives...
elif action=='2':
    print("Please enter the first or last name of a representative, and capitalize the first letter:")
    name = input()
    mask = df[df['representative'].str.contains(name)]
    mask
    if len(mask.index)==0:
        print("Error: No representative with this first or last name is listed in the dataset. Make sure your capitalization is correct") #Error notification
    else:
        print(mask) ##Returns information for selected representative(s)
        shape=mask.shape #Benchmark i5: output brief summary of the data file ingestion after it has processed and include number of records and columns
        print('Number of columns:', shape[1])
        print('Number of records:', shape[0])
        #Benchmark i2: Convert the general format and data structure of the data source to any target (user selects between csv and sql output type)
        output_type = str(input("Output csv or sql?:"))
        if output_type=="csv": ##If the user selects output as csv
            mask.to_csv("select_representatives.csv") #Benchmark i4: Converted (new) file is written to disk
            print("File sucessfully downloaded as: select_representatives.csv") #Confirms output success
        elif output_type=="sql": ##If the user selects output as sql
            conn=sqlite3.connect("select_representatives.db")
            cur=conn.cursor()
            cur.execute("CREATE TABLE IF NOT EXISTS select_representatives (disclosure_year number, disclosure_date text, transaction_date text, owner text, ticker text, asset_description text, type text, amount text, representative text, district text, ptr_link text, cap_gains_over_200_usd number)")
            conn.commit()
            df.to_sql("shows", conn, if_exists="replace", index=False)
            cur.execute('''
            SELECT * FROM select_representatives
            ''') #Benchmark i4: Converted (new) file is written to disk
            print("File sucessfully downloaded as: select_representatives.sql") ##Confirms output success
        else:
            print("Error outputting. Please enter 'csv' or 'sql'. Check your capitalization and spelling") ##Output error notification
elif action=='3':
    print("Please enter a stock ticker, and capitalize all letters:")
    ticker = input()
    mask = df[df['ticker'].str.contains(ticker)]
    mask
    if len(mask.index)==0:
        print("Error: No stock with this ticker is listed in the dataset. Make sure your capitalization is correct")
    else:
        print(mask) ##Returns information for selected stock ticker
        shape=mask.shape  #Benchmark i5: output brief summary of the data file ingestion after it has processed and include number of records and columns
        print('Number of columns:', shape[1])
        print('Number of records:', shape[0])
        #Benchmark i2: Convert the general format and data structure of the data source to any target (user selects between csv and sql output type)
        output_type = str(input("Output csv or sql?:"))
        if output_type=="csv": ##If the user selects output as csv
            mask.to_csv("select_tickers.csv") #Benchmark i4: Converted (new) file is written to disk
            print("File sucessfully downloaded as: select_tickers.csv") #Confirms output success
        elif output_type=="sql": ##If the user selects output as sql
            conn=sqlite3.connect("select_tickers.db")
            cur=conn.cursor()
            cur.execute("CREATE TABLE IF NOT EXISTS select_tickers (disclosure_year number, disclosure_date text, transaction_date text, owner text, ticker text, asset_description text, type text, amount text, representative text, district text, ptr_link text, cap_gains_over_200_usd number)")
            conn.commit()
            df.to_sql("shows", conn, if_exists="replace", index=False)
            cur.execute('''
            SELECT * FROM select_tickers
            ''') #Benchmark i4: Converted (new) file is written to disk
            print("File sucessfully downloaded as: select_tickers.sql") ##Confirms output success
        else:
            print("Error outputting. Please enter 'csv' or 'sql'. Check your capitalization and spelling") #Output error notification
elif action=='4':
    print("Please enter a disclosure year:")
    year = input()
    mask =  df[df['disclosure_year'].astype(str).str.contains(year)]
    mask
    if len(mask.index)==0:
        print("Error: The year you have entered is not listed as a disclosure year in the dataset")
    else:
        print(mask) ##Returns information for selected disclosure year
        shape=mask.shape  #Benchmark i5: output brief summary of the data file ingestion after it has processed and include number of records and columns
        print('Number of columns:', shape[1])
        print('Number of records:', shape[0])
        #Benchmark i2: Convert the general format and data structure of the data source to any target (user selects between csv and sql output type)
        output_type = str(input("Output csv or sql?:"))
        if output_type=="csv": ##If the user selects output as csv
            mask.to_csv("select_disclosure_years.csv") #Benchmark i4: Converted (new) file is written to disk
            print("File sucessfully downloaded as: select_disclosure_years.csv") ##Confirms output success
        elif output_type=="sql": ##If the user selects output as sql
            conn=sqlite3.connect("select_disclosure_years.db")
            cur=conn.cursor()
            cur.execute("CREATE TABLE IF NOT EXISTS select_disclosure_years (disclosure_year number, disclosure_date text, transaction_date text, owner text, ticker text, asset_description text, type text, amount text, representative text, district text, ptr_link text, cap_gains_over_200_usd number)")
            conn.commit()
            df.to_sql("shows", conn, if_exists="replace", index=False)
            cur.execute('''
            SELECT * FROM select_disclosure_years
            ''') #Benchmark i4: Converted (new) file is written to disk
            print("File sucessfully downloaded as: select_disclosure_years.sql") ##Confirms output success
        else:
            print("Error outputting. Please enter 'csv' or 'sql'. Check your capitalization and spelling") ##Output error notification
elif action=='5':
    print("Please enter a district:")
    district = str(input())
    mask = df[df['district'].str.contains(district)]
    mask
    if len(mask.index)==0:
        print("Error: The district you have entered is listed in the dataset. Make sure your capitalization is correct")
    else:
        print(mask) ##Returns information for selected district
        shape=mask.shape  #Benchmark i5: output brief summary of the data file ingestion after it has processed and include number of records and columns
        print('Number of columns:', shape[1])
        print('Number of records:', shape[0])
        #Benchmark i2: Convert the general format and data structure of the data source to any target (user selects between csv and sql output type)
        output_type = str(input("Output csv or sql?:"))
        if output_type=="csv": ##If the user selects output as csv
            mask.to_csv("select_districts.csv") #Benchmark i4: Converted (new) file is written to disk
            print("File sucessfully downloaded as: select_districts.csv") ##Confirms output success
        elif output_type=="sql": ##If the user selects output as sql
            conn=sqlite3.connect("select_districts.db")
            cur=conn.cursor()
            cur.execute("CREATE TABLE IF NOT EXISTS select_districts (disclosure_year number, disclosure_date text, transaction_date text, owner text, ticker text, asset_description text, type text, amount text, representative text, district text, ptr_link text, cap_gains_over_200_usd number)")
            conn.commit()
            df.to_sql("shows", conn, if_exists="replace", index=False)
            cur.execute('''
            SELECT * FROM select_districts
            ''') #Benchmark i4: Converted (new) file is written to disk
            print("File sucessfully downloaded as: select_districts.sql") #Confirms output success
        else:
            print("Error outputting. Please enter 'csv' or 'sql'. Check your capitalization and spelling") #Output error notification
elif action=='6':
    names = np.unique(df['representative'])
    print(names)
elif action=='7':
    ticker = np.unique(df['ticker'])
    print(ticker)
elif action=='8':
    year = np.unique(df['disclosure_year'])
    print(year)
elif action=='9':
    district = np.unique(df['district'])
    print(district)
else:
    print("Error: Please input a number 1 through 9 to complete the requested action")

Please enter a number 1 through 9 to select an action:2
Please enter the first or last name of a representative, and capitalize the first letter:
Pelosi
       disclosure_year disclosure_date transaction_date owner ticker  \
535               2020      09/03/2020       2020-08-07  self     FB   
3067              2021      04/09/2021       2021-03-19  self   MSFT   
3068              2021      04/09/2021       2021-03-19  self   MSFT   
3069              2021      04/09/2021       2021-03-10  self   RBLX   
3470              2021      03/09/2021       2021-02-18  self     AB   
...                ...             ...              ...   ...    ...   
14761             2020      04/01/2020       2020-02-27  self  GOOGL   
14762             2020      04/01/2020       2020-02-28  self   MSFT   
14763             2020      04/01/2020       2020-02-20  self   MSFT   
14764             2020      04/01/2020       2020-02-21  self   MSFT   
14765             2020      04/01/2020       2020-02-20